# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [85]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [86]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [87]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [88]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra database and table setup 

#### Creating a Cluster

In [89]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [90]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [91]:
# Connect to Keyspace

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Task 1: Getting artist, song title an song length by session and item

#### Drop table if it already exists in order to adjust columns etc more flexible

In [92]:
# Drop Table if it already exists
query = "DROP TABLE IF EXISTS song_plays_by_session "
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Create table
For this query session_id and item_in_session are again considered as composite key in order to be able to query song information by session and item in this session.

In [93]:
# Create Table
query = "CREATE TABLE IF NOT EXISTS song_plays_by_session "
query = query + "(session_id int, item_in_session int, artist_name text, song_name text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Insert data into the table

In [94]:
# read csv data and insert into table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_plays_by_session (session_id, item_in_session, artist_name, song_name, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### Select the task data from the table

In [95]:
# select data from table

query = "SELECT artist_name, song_name, song_length FROM song_plays_by_session WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Task 2: Getting list of artist, song title and user by session and user orderd by order of play

#### Drop table if it already exists in order to adjust columns etc more flexible

In [96]:
# Drop Table
query = "DROP TABLE IF EXISTS song_plays_by_user "
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Create table
Create table to query songs played by a user in a session ordered in playing sequence. Therefore session_id and user_id form the composite key and item_in_session orders the data as the clustering key.

In [97]:
# Create Table
query = "CREATE TABLE IF NOT EXISTS song_plays_by_user "
query = query + "(session_id int, user_id int, item_in_session int, user_first_name text, user_last_name text,  artist_name text, song_name text, PRIMARY KEY ((session_id, user_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Insert data into the table

In [98]:
# read csv data and insert into table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_plays_by_user (session_id, user_id, item_in_session, user_first_name, user_last_name, artist_name, song_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[10]),int(line[3]),line[1],line[4],line[0],line[9]))

#### Select the task data from the table

In [99]:
# select data from table

query = "SELECT artist_name, song_name, user_first_name, user_last_name FROM song_plays_by_user WHERE session_id = 182 and user_id = 10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Task 3: Get the list of unique users that played a specific song

#### Drop table if it already exists in order to adjust columns etc more flexible

In [100]:
# Drop Table
query = "DROP TABLE IF EXISTS users_by_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Create table
Create table with songs and all unique users that ever played this song. Song_name and user_id act as composite key in order to catch all songs and their unique users. By using user_id instead of the user names we can avoid creating duplicates when users share same name.

In [101]:
# Create Table
query = "CREATE TABLE IF NOT EXISTS users_by_song"
query = query + "(song_name text, user_id int, user_first_name text, user_last_name text, PRIMARY KEY (song_name, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Insert data into the table

In [102]:
# read csv data and insert into table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (user_id, song_name, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]),line[9],line[1],line[4]))

#### Select the task data from the table

In [103]:
# select data from table

query = "SELECT user_first_name, user_last_name FROM users_by_song WHERE song_name = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [104]:
query = "drop table if exists song_plays_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [105]:
query = "drop table if exists song_plays_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [106]:
query = "drop table if exists users_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [107]:
session.shutdown()
cluster.shutdown()